In [43]:
# encoding: utf-8
# encoding: iso-8859-1
# encoding: win-1252
import json
import nltk
from nltk.tokenize import TweetTokenizer
from IPython.display import clear_output, display
from nltk.corpus import stopwords
import re
import math


tknzr = TweetTokenizer()
with open('../inverted_index/inverted_index.json') as f:
    inverted_index = json.load(f)
    

stopwords_en = set(stopwords.words('english'))
stopwords_pt = nltk.corpus.stopwords.words('portuguese')
especial_c = ["-",",",";","'",")","(",":", ".","’","–","","...",
              "eua","estados","unidos","filme","reino","brasil","franca",
              "espanha","unido","japao","alemanha","italia","suecia","canada"]




### Dicionario de titulo, autor e elenco

In [44]:
titulo = {}
direcao = {}
elenco = {}


def converter_string(t):
    t = t.lower()  
    t= t.replace("ã©","é")
    t = t.replace("ã§","ç")
    t= t.replace("ãº","ú")
    t= t.replace("ã¡","á")
    t=t.replace("ã£","ã")
    t=t.replace("ãª","ê")
    t= t.replace("ã³","ó")
    t =t.replace("ãµ","õ")
    t =t.replace("ã´","ô")
    t = t.replace("ã\xad","í")
    t = re.sub('\W+','', t )
    t = ''.join([i for i in t if not i.isdigit()])
    
    return t

nome = ""
inverted_index['titulo.pele']
for obj in inverted_index:
    if("titulo." in obj):
        t = obj
        t = t.replace("titulo.","")
        t = converter_string(t)
        if (t not in stopwords_en) and (t not in stopwords_pt) and (t not in especial_c):
            titulo[t] = inverted_index[obj][1]
    if("direcao." in obj):
        t = obj
        t = t.replace("direcao.","")
        t = converter_string(t)
        if (t not in stopwords_en) and (t not in stopwords_pt) and (t not in especial_c):
            direcao[t] = inverted_index[obj][1]
    if("elenco." in obj):
        t = obj
        t = t.replace("elenco.","")
        t = converter_string(t)
        if (t not in stopwords_en) and (t not in stopwords_pt) and (t not in especial_c):
            elenco[t] = inverted_index[obj][1]

print(len(elenco))
print(len(titulo))
print(len(direcao))

18104
3742
2243


### Dicionario com as probabilidades de cada palavra

In [45]:
prob_titulo = {}
prob_direcao = {}
prob_elenco = {}

qtd_titulos = len(titulo)
for key, valor in titulo.items():
    prob_titulo[key] = len(valor)/qtd_titulos

qtd_direcao = len(direcao)
for key, valor in direcao.items():
    prob_direcao[key] = len(valor)/qtd_direcao

qtd_elenco = len(elenco)
for key, valor in elenco.items():
    prob_elenco[key] = len(valor)/qtd_elenco

#prob_direcao

In [40]:
# import operator
# sortede = sorted(prob_titulo.items(), key=operator.itemgetter(1),reverse=True)
# sortede

2244

In [46]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [ ]:
%%cython
from __main__ import *


def mutual_info(dicionario, prob_list):
    retorno = {}
    independente = 1
    cont = 0
    keys = list(dicionario.keys())
    tamanho = len(keys)
    for i in range(tamanho):
        cont = cont+1
        clear_output(wait=True)
        print("porcentagem: " + str(((cont/tamanho)*100)))
        key_a = keys[i]
        val_a = dicionario[key_a]
        for j in range(i+1,tamanho):
            key_b = keys[j]
            val_b = dicionario[key_b]
            for k in range(j+1,tamanho):
                key_c = keys[k]
                val_c = dicionario[key_c]
                for a in val_a:
                    independente = 1
                    for b in val_b:
                        if b == a:
                            for c in val_c:
                                if c == a:
                                    t = (key_a, key_b, key_c)
                                    #print(t)                    
                                    if t in retorno:
                                        retorno[t] = retorno[t] +1
                                    else:
                                        retorno[t] = 1
                                if c > a:
                                    break
                        if b > a:
                            break
    for n in retorno:
        retorno[n] = math.log2(retorno[n]/(prob_list[n[0]]*prob_list[n[1]]*prob_list[n[2]]))

    return retorno

direcao_mi = mutual_info(direcao, prob_direcao)
titulo_mi  = mutual_info(titulo, prob_titulo)
elenco_mi  = mutual_info(elenco, prob_elenco) 


porcentagem: 0.7579135086937138


In [ ]:
# import math
# m_info = {}
# for n in probs:
#     m_info[n] = math.log2(prob_nomes[n[0]]*prob_nomes[n[1]]*prob_nomes[n[2]])

# final = dict(m_info)
# final.update(probs_idp)

In [ ]:
# m_info
# sorted_y = sorted(m_info.items(), key=operator.itemgetter(1),reverse=True)
# sorted_y